<a href="https://colab.research.google.com/github/Bitang-Melyen-Tanulok/Csip_Csip/blob/main/Cross_valid_Train_AAAAAAA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip install numpy
!pip install matplotlib
!pip install tensorflow
!pip install sklearn
!pip install google

   ---------------------------------------- 0.0/8.0 MB ? eta -:--:--
   ------ --------------------------------- 1.3/8.0 MB 7.4 MB/s eta 0:00:01
   -------------- ------------------------- 2.9/8.0 MB 7.3 MB/s eta 0:00:01
   ---------------------- ----------------- 4.5/8.0 MB 7.5 MB/s eta 0:00:01
   ------------------------------- -------- 6.3/8.0 MB 7.6 MB/s eta 0:00:01
   ---------------------------------------  7.9/8.0 MB 7.7 MB/s eta 0:00:01
   ---------------------------------------- 8.0/8.0 MB 7.5 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.2 MB ? eta -:--:--
   ---------------------------- ----------- 1.6/2.2 MB 9.3 MB/s eta 0:00:01
   ---------------------------------------- 2.2/2.2 MB 8.3 MB/s eta 0:00:00
   ---------------------------------------- 0.0/2.6 MB ? eta -:--:--
   --------------------------- ------------ 1.8/2.6 MB 8.4 MB/s eta 0:00:01
   ---------------------------------------- 2.6/2.6 MB 8.4 MB/s eta 0:00:00
   ------------------------------

  You can safely remove it manually.
  You can safely remove it manually.


  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [12]:
!pip install sklearn

  Using cached sklearn-0.0.post12.tar.gz (2.6 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [15 lines of output]
  The 'sklearn' PyPI package is deprecated, use 'scikit-learn'
  rather than 'sklearn' for pip commands.
  
  Here is how to fix this error in the main use cases:
  - use 'pip install scikit-learn' rather than 'pip install sklearn'
  - replace 'sklearn' by 'scikit-learn' in your pip requirements files
    (requirements.txt, setup.py, setup.cfg, Pipfile, etc ...)
  - if the 'sklearn' package is used by one of your dependencies,
    it would be great if you take some time to track which package uses
    'sklearn' instead of 'scikit-learn' and report it to their issue tracker
  - as a last resort, set the environment variable
    SKLEARN_ALLOW_DEPRECATED_SKLEARN_PACKAGE_INSTALL=True to avoid this error
  
  More information is available at
  https://github.com/scikit-learn/sklearn-pypi-package
  [end of output]
  
  note: This error originates from a subpr

In [14]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing import image
from sklearn.preprocessing import LabelEncoder
from google.colab import drive

X = []
Y = []
drive.mount('/content/drive')
path = '/content/drive/MyDrive/DeepLearning'

spectrograms_path = os.path.join(path, 'ugyanaz?')  # Path to folder with training data
folders = os.listdir(spectrograms_path)

for folder in folders:
    print(f"Current folder: {folder}")
    folder_path = os.path.join(spectrograms_path, folder)

    # Reading spectrogram files
    files = os.listdir(folder_path)

    for file in files:
        if file.endswith('.png'):  # Only reading PNGs
            file_path = os.path.join(folder_path, file)

            # Loading image
            img = image.load_img(file_path, color_mode='grayscale')
            img_array = image.img_to_array(img)  # Converting image into array
            X.append(img_array)

            # Adding label based on the folder name
            Y.append(folder)

X = np.array(X)
Y = np.array(Y)

# Print X and Y shape
print(f"X shape: {X.shape}")
print(f"Y shape: {Y.shape}")

# Shuffle X and Y the same way
permutation = np.random.permutation(len(X))
X = X[permutation]
Y = Y[permutation]

ModuleNotFoundError: No module named 'sklearn'

Since this is a multi-class classification task, I am converting labels to one-hot format:

In [ ]:
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

#First, the labels need to be converted into numerical values
le = LabelEncoder()
Y_encoded = le.fit_transform(Y)

#Getting number of classes
num_classes = len(le.classes_)
print(f"Class number= {num_classes}")

#Converting to one-hot encoding
Y_onehot = to_categorical(Y_encoded, num_classes)

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

#Implementing early stopping, since there is no reason for it to learn further when val_loss isn't decreasing
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5, #If it doesn't improve for 5 epochs, it concludes
    verbose=1,
    restore_best_weights=True
)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

#Wanting to save the best model, so implementing checkpointing
checkpoint = ModelCheckpoint(
    'best_model.keras',
    monitor='val_loss',
    save_best_only=True,
    mode='min',
    verbose=1
)

In [ ]:
from tensorflow.keras import layers, models
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, BatchNormalization
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

fold_accuracies = []
fold_losses = []

history_per_fold = []

# Start Stratified K-Fold cross-validation
fold_number = 1
for train_index, valid_index in skf.split(X, Y_encoded):
  print(f"Startung fold: {fold_number}")

  X_train, X_valid = X[train_index], X[valid_index]
  Y_train, Y_valid = Y_onehot[train_index], Y_onehot[valid_index]

  X_train = X_train / np.max(X_train)
  X_valid = X_valid / np.max(X_valid)

  model = models.Sequential()
  model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2], 1)))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(64, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Conv2D(128, (3, 3), activation='relu'))
  model.add(layers.MaxPooling2D((2, 2)))
  model.add(layers.Flatten())
  model.add(layers.Dense(128, activation='relu'))
  model.add(layers.Dropout(0.5))
  model.add(layers.Dense(num_classes, activation='softmax'))

  model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

  history = model.fit(
      X_train, Y_train,
      epochs=50,
      batch_size=32,
      validation_data=(X_valid, Y_valid),
      callbacks=[early_stopping, checkpoint],
      verbose=1
  )

  history_per_fold.append(history)

  valid_loss, valid_accuracy = model.evaluate(X_valid, Y_valid)

  fold_accuracies.append(valid_accuracy)
  fold_losses.append(valid_loss)

  fold_number += 1

model.summary()

In [ ]:
print("\nCross-Validation Results:")
print(f"Mean Validation Loss: {np.mean(fold_losses)}")
print(f"Mean Validation Accuracy: {np.mean(fold_accuracies)}")

for i, (loss, accuracy) in enumerate(zip(fold_losses, fold_accuracies), start=1):
    print(f"Fold {i} - Validation Loss: {loss}, Validation Accuracy: {accuracy}")

In [ ]:
# Plot loss and accuracy for each fold
for i, history in enumerate(history_per_fold):
  plt.figure(figsize=(12, 6))

  # Plot accuracy
  plt.subplot(1, 2, 1)
  plt.plot(history.history['accuracy'], label='Training Accuracy')
  plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
  plt.title(f'Fold {i + 1} Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy')
  plt.legend()

  # Plot loss
  plt.subplot(1, 2, 2)
  plt.plot(history.history['loss'], label='Training Loss')
  plt.plot(history.history['val_loss'], label='Validation Loss')
  plt.title(f'Fold {i + 1} Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  plt.tight_layout()
  plt.show()